In [ ]:
"""
1. CÁC THUẬT NGỮ

- Đơn vị(unit): là giá trị của 1 điểm nằm trên ma trận khối ở mỗi tầng của mạng CNN.

- Vùng nhận thức(Receptive Field): Là vùng ảnh trên khối ma trận đầu vào mà bộ loc 
  sẽ nhận tích chập để ánh xạ tới một đơn vị trên layer tiếp theo.

- Vùng địa phương(Local Region): Theo một ngĩa nào đó sẽ bao hàm cả vùng nhận 
  thức. Là vùng ảnh cụ thể nằm trên khối ma trận ở các tầng layer của mạng CNN.

- Bản đồ đặc trưng(Feature Map): Là ma trận đầu ra khi áp dụng phép tích chập từ
  giữa bộ lọc với với các vùng nhận thức theo phương di chuyển từ trái qua phải
  và từ trên xuống dưới.

- Bản đồ kích hoạt(Activation Map): La output của bản đồ đặc trưng CNN khi áp
  dụng thêm hàm #activation để tính đạo hàm phi tuyến

2. KIẾN TRÚC CỦA MẠNG NEURAL TÍCH CHẬP
- về cơ bản 1 mạng của neural tích chập 2 chiều có dạng như sau:
  
  intput -> [[conv -> RELU]*N -> POOL?]*M -> [FC -> RELU]*K -> FC

- Trong đó:
  + input: tầng đầu vào
  + conv: tầng tích chập
  + RELU: Tầng kích hoạt. Thông qua hàm kích hoạt (activation function), thường la ReLU
  + POOL: Tầng tổng hợp, thông thường là MaxPooling 
  + FC: Fully connected layer- tầng kết nối hoàn toàn. Thông thường tầng này ở cuối 
    cùng và kết nối với các đơn vị đại diện cho nhóm phân loại   
  + M, N, K: số lần lặp lại

  => Mạng Neural có 3 quá trình khác nhau: 
  - Quá trình tích chập - Convoluion
  - Quá trình tổng hợp - MaxPooling
  - Quá trình kết nối hoàn toàn - Fully connected

3. Tính chất cảu mạng Neural tích chập
- Tính kết nối trượt: kết nối từng vùng địa phương(local region) hoặc vùng nhận thức (receptive field)
  có kích thích bằng với kích thước bộ lọc của hình ảnh đó- TRƯỢT TỪ TRÁI QUA PHẢI, TỪ TRÊN XUỐNG DƯỚI 

- Các khối Neural 3D được sắp xếp 1 cách hợp lý theo 3 chiều width, height, depth(sâu)

- Tính chia sẻ kết nối và kết nối cục bộ: Không kết nối trực tiếp toàn bộ khối 3D
  mà sẽ chọn ra các #vùng địa phương(vùng nhận thức) có kích thước bằng với bộ lọc 

"""



In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import logging 


In [1]:



def cnn_model_fn(feature, labels, mode):
    """ MOdel function for CNN """
    #input layer
    input_layer = tf.reshape(feature['x'], shape=[-1,28,28,1])


    #Convolution layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size = [5,5],
        padding = 'same',
        activation = tf.nn.relu
    )
        #Apply formula:N1 = (N+2P-f)/S + 1
        #in which: N is input image size, P is padding size, f is filter size and S is step


        #Output tensor shape: N1 = (28-5)/1+1 = 24 => shape = [-1, 24, 24, 1]
        #But we at parameter we set padding = 'same' in order to keep output shape unchange to input shape 
        #Thus output shape is [-1, 28, 28, 1]
    
    #Max pooling layer 1
    pool1 = tf.layers.max_pooling2d(
        inputs = conv1,
        pool_size = [2,2],
        strides = 2
    )

    #Convolution layer 2
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 64,
        kernel_size = [5,5],
        padding = 'same',
        activation = tf.nn.relu
    )

    #Max pooling 2
    pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = [2,2],
        strides = 2
    )
    #Output tensor shape: N4 = (14-2)/2+1 = 7 => shape = [-1, 7, 7, 1]
    
    #Dense layer
    flat = tf.reshape(pool2, [-1, 7*7*64])
    dense = tf.layers.dense(
        inputs = flat,
        units = 1024,
        activation = tf.nn.relu
    )

    dropout = tf.layers.dropout(
        inputs = dense,
        rate = 0.4,
        training = mode == tf.estimator.ModeKeys.TRAIN
    )

    #Logits layer
    logits = tf.layers.dense(_Inputs = dropout, units = 10)

    predictions = {
        'classes': tf.argmax(input = logits, axis= 1, name = 'class_tensor'),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits = logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learining_rate = 0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(mode=mode, loss = loss, train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(
                labels = labels, predictions = predictions['classes']
            )
        }
        return tf.estimator.EstimatorSpec(
            mode = mode, loss = loss, eval_metric_ops=eval_metric_ops
        )

In [ ]:
import sys
from mnist import MNIST
from imutils import path

In [ ]:
mndata = MNIST('../input')
mndata.load_training()
train_data = np.asarray(mndata.train_images)/255.0
train_labels = np.array(mndata.test_labels.tolist)

FileNotFoundError: ignored